# Example for inspecting tensors from output files
## Requirements:
1. python >= 3
2. numpy
3. keep the `tensor_output_reader.py` in the same folder as this notebook.
4. Some output file to read, with tensor(s) printed out.

In [ ]:
import numpy as np
import tensor_output_reader as tor
import sys
from pathlib import Path

p = Path.cwd()
for _ in range(5):
    if p.name == "main" and (p / "__init__.py").exists():
        print(p.name)
        print(sys.path)
        sys.path.insert(0, str(p))
        break
    p = p.parent
    
import tabulate_outs as to
np.set_printoptions(
    linewidth=np.inf
)
np.set_printoptions(suppress=True,floatmode='maxprec') # just for readability

## Grab tensor as numpy array

### C++ tensor

In [ ]:
# Grab tensor
outfile = 'comp_between_default_and_standard_cu1/standard_check_for_cpp_printing.193285-20251216.out'
vf = tor.grab_tensor_from_std(outfile, 'VF[mnij]')
print(vf.shape)
std_ener = to.get_ener(outfile.rstrip('.out') + '.xml')

### Fortran tensor (convert to full to get the full i,j,k,l indices)

In [ ]:
outfile = 'comp_between_default_and_standard_cu1/default_fortran_printing.193283-20251216.out'
vff_reduced = tor.grab_tensor_from_def(outfile, 'vmat')
vff = tor.convert_to_full(vff_reduced, vf.shape)
print(vff.shape)
def_ener = to.get_ener(outfile.rstrip('.out') + '.xml')

In [ ]:
outfile = 'comp_between_default_and_standard_cu1/default_nofix.193284-20251216.out'
vffnof_reduced = tor.grab_tensor_from_def(outfile, 'vmat')
vffnof = tor.convert_to_full(vffnof_reduced, vf.shape)
print(vffnof.shape)
defnofix_ener = to.get_ener(outfile.rstrip('.out') + '.xml')

In [ ]:
print(std_ener, def_ener, defnofix_ener)

## visualize in the same format as output file (for verification)

In [ ]:
display(vf[:,:,2,1]) # --> should match first "Block [ i j 2 1 ]" in output file
print("====")
display(vff[:,:,2,1])
print("====")
display(vffnof[:,:,2,1])

## Iterate through tensor and print index, value

In [ ]:
len(index)

In [ ]:
for index, value in np.ndenumerate(vf):
    if len(set(index)) == len(index): continue
    if value != 0:
        value_str = f"{value:15.11f}"
    else:
        value_str = f"{0:^15d}" # just for readability

    # Optionally, also add other tensors for comparison:
    value2 = vffnof[index]
    if value2 != 0:
        value2_str = f"{value:15.11f}"
    else:
        value2_str = f"{0:^15d}" # just for readability
        
    print(index, value_str, value2_str)